In [ ]:
!pip install gradio torch Pillow numpy torchvision

import torch
import numpy as np
from PIL import Image
import gradio as gr
import matplotlib.pyplot as plt
from torchvision.transforms import v2
from torch import nn

class CNN(nn.Module):
    def __init__(self, hidden_dim=256, chan_dim=32, normalize=nn.BatchNorm2d, activation=nn.GELU, dropout=0.2):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, chan_dim, kernel_size=5),
            activation(),
            normalize(chan_dim),
            nn.Conv2d(chan_dim, chan_dim, kernel_size=5),
            nn.MaxPool2d(2),
            activation(),
            normalize(chan_dim),
            nn.Conv2d(chan_dim, 2 * chan_dim, kernel_size=5),
            nn.MaxPool2d(2),
            activation(),
            normalize(2 * chan_dim),
        )
        self.head = nn.Sequential(
            nn.Linear(18 * chan_dim, hidden_dim),
            activation(),
            nn.BatchNorm1d(hidden_dim),
            nn.Dropout(p=dropout),
            nn.Linear(hidden_dim, 10)
        )
    
    def forward(self, x):
        batch, _, _, _ = x.shape
        x = self.model(x)
        x = x.reshape(batch, -1)
        x = self.head(x)
        return x


model = CNN()
model.load_state_dict(torch.load('weights.pt'))
model.eval()

def predict_image(img):
    img = Image.fromarray(img.astype('uint8')).convert('L')
    img_array = np.array(img) / 255.0
    img_tensor = torch.FloatTensor(img_array).unsqueeze(0).unsqueeze(0)
    img_tensor = v2.Resize((28, 28))(img_tensor)
    img_tensor = v2.Grayscale()(img_tensor)
    with torch.no_grad():
        output = model(img_tensor)
        probs = torch.softmax(output, dim=-1)[0]
        pred_class = torch.argmax(probs).item()
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
    
    ax1.imshow(img_array, cmap='gray')
    ax1.set_title(f"Предсказан класс: {pred_class}")
    ax1.axis('off')
    
    ax2.bar(range(10), probs.numpy(), color='skyblue')
    ax2.set_xticks(range(10))
    ax2.set_xticklabels(list(range(10)), rotation=45)
    ax2.set_title('Вероятности классов')
    ax2.set_ylim(0, 1)
    
    return fig


demo = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(label="Загрузите изображение символа"),
    outputs=gr.Plot(label="Результат классификации"),
    title="Классификатор KMNIST",
    description="Загрузите изображение японского символа для классификации"
)

demo.launch(share=False, inline=True)
